In [ ]:
import os
import ast
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.join(os.getcwd(), '../desi/'))
import catalog_definitions as cat
import numpy as np

In [ ]:
# Folder number to look in
folder_num = 6

models = []
chi = []
with open(f'mcmc_{folder_num}/out.{folder_num}', 'r') as file:
    model_num = -1
    for line in file:
        if line.startswith('MODEL'):
            model_num = int(line.split(' ', 2)[-1].strip())
        elif line.startswith('{\'zmin\':'):
            parameter_dict = ast.literal_eval(line)
            models.append(parameter_dict)
        elif line.startswith('CHI'):
            chi.append(float(line.split(' ', 2)[-1].strip()))

    assert len(models) == len(chi)

def get_parameter_values(parameter_name):
    return [model[parameter_name] for model in models]

exclusions = ['zmin', 'zmax', 'frac_area', 'fluxlim', 'color']
for name in models[0]:
    if name in exclusions:
        continue
    plt.plot(get_parameter_values(name))
    plt.xlabel('Iteration')
    plt.ylabel(f'Parameter Value')
    plt.title(f'Parameter Chain for {name}')
    plt.show()

plt.plot(chi)
plt.xlabel('Iteration')
plt.ylabel('Chi Squared')
plt.title('Chi Squared Chain')
plt.show()

# Show best model
best_model = models[chi.index(min(chi))]
print(f'Best model is model {chi.index(min(chi))} with chi squared of {min(chi)}')

#compare each property of best model to cat.sdss_colors.GF_props
for key in best_model:
    if key in exclusions:
        continue
    print(f'{key.ljust(12)}:  {best_model[key]:.4} vs {cat.sdss_colors.GF_props[key]:.4} ({100 * (best_model[key] - cat.sdss_colors.GF_props[key]) / cat.sdss_colors.GF_props[key]:.2}%)')


In [ ]:
# TODO

# --- plot chain
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
#samples = sampler.get_chain()
samples = sampler.flatchain
#print samples.shape
labels = ["m", "b", "log(f)"]
for i in range(3):
    ax = axes[i]
    ax.plot(samples[:, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")
#plt.show()